In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
pd.options.display.max_rows = 6000
import keras

In [2]:
X = pd.read_csv('data_with_weather_wo_outliers.csv')
X = X.set_index('Date')
X.index = pd.to_datetime(X.index)
y = X.loc[:, 'total_demand'].copy()
X = X.drop(['total_demand'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [3]:
# Scale the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input data for LSTM
#X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
#X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [13]:
X_train.shape

(632448, 72)

In [10]:
model = Sequential()
model.add(Conv1D(64, (3), activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(15, (3)))
model.add(LSTM(units=256, activation='relu', return_sequences=True))
model.add(LSTM(units=256, activation='relu', return_sequences=True))
model.add(LSTM(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
optimizer = keras.optimizers.Adam()
model.compile(loss='mape', optimizer=optimizer)

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 76, 64)            256       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 21, 64)           0         
 1D)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 21, 256)           328704    
                                                                 
 lstm_7 (LSTM)               (None, 21, 256)           525312    
                                                                 
 lstm_8 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                      

In [12]:
history = model.fit(X_train, y_train, epochs=50, batch_size=1000, validation_data=(X_test, y_test))

Epoch 1/50
633/633 [==============================] - 2595s 4s/step - loss: 39.9154 - val_loss: 41.0890
Epoch 2/50
633/633 [==============================] - 2388s 4s/step - loss: 28.8258 - val_loss: 38.3533
Epoch 3/50
633/633 [==============================] - 2301s 4s/step - loss: 28.0296 - val_loss: 37.0706
Epoch 4/50
633/633 [==============================] - 2295s 4s/step - loss: 27.4787 - val_loss: 32.9211
Epoch 5/50
633/633 [==============================] - 2254s 4s/step - loss: 26.8375 - val_loss: 39.1778
Epoch 6/50
633/633 [==============================] - 2278s 4s/step - loss: 26.1527 - val_loss: 50.2494
Epoch 7/50
633/633 [==============================] - 2248s 4s/step - loss: 24.2001 - val_loss: 54.6222
Epoch 8/50
633/633 [==============================] - 2236s 4s/step - loss: 21.7748 - val_loss: 52.4430
Epoch 9/50
633/633 [==============================] - 2246s 4s/step - loss: 22.8189 - val_loss: 43.9002
Epoch 10/50
633/633 [==============================] - 2231s 4s/

KeyboardInterrupt: 